In [ ]:
# https://siqi-zhu.medium.com/ldavis-a-deep-dive-into-the-popular-topic-modeling-tool-d0c61a03e969

In [74]:
from __future__ import print_function
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

from collections import Counter
from io import StringIO
from datetime import datetime
from opencc import OpenCC
import pandas as pd
import requests
import jieba
import re
import os

import pyLDAvis.lda_model
import pyLDAvis

cc = OpenCC('s2tw')

In [75]:
pyLDAvis.__version__

'3.4.0'

In [76]:
pd.__version__

'2.1.2'

In [77]:
df = pd.read_excel("./system_problem.xlsx")
df.head()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266 entries, 0 to 265
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   開單時間    266 non-null    datetime64[ns]
 1   機關      266 non-null    object        
 2   回覆時間    266 non-null    datetime64[ns]
 3   摘要      266 non-null    object        
 4   細節      266 non-null    object        
 5   回覆      266 non-null    object        
 6   value   266 non-null    object        
 7   問題單分類   266 non-null    object        
 8   單號      266 non-null    object        
 9   主機關代碼   266 non-null    int64         
dtypes: datetime64[ns](2), int64(1), object(7)
memory usage: 20.9+ KB


In [78]:
# 本地 csv 文件，可以是本地文件，也可以是遠程文件
# source_csv_path = 'CRM_reviews.csv'
# 文本 csv 文件裡面文本所處的列名,注意這裡一定要填對，要不然會報錯的！

document_column_name = '摘要'
# 輸出主題詞的文件路徑
top_words_csv_path = 'poetry_topic_modeling.csv'
# 輸出各文檔所屬主題的文件路徑
predict_topic_csv_path = 'poetry-distribution.csv'
# 可視化 html 文件路徑
html_path = 'poetry-lda-visualization.html'
# 選定主題數
n_topics = 3
# 要輸出的每个主題的前 n_top_words 主題詞數
n_top_words = 1500
# 去除無意義字符的正則表達式
pattern = r'[\\s,.<>/?:;\'\"[\\]{}()\\|~!\t"@#$%^&*\\-_=\\+，。\n《》、？：；“”‘’｛｝【】（）…￥！—┄－]+'
# 停頓詞定義
# stop = r'我|你'



In [79]:
df["摘要"]

0                         人事室主任郭燕燕要查看輪班相關資料，經查詢無資料，畫面如附件
1                                        有關北嶺國小112年寒暑休設定
2                                        秘書室新進人員無法選取該室職稱
3             1/3及1/4曠職部分麻煩請處理，因該員原任職台東地院1/3辭職，1/12本院報到。
4                  1/10請假，值班表設定帶頒給陳妘綺，申請完成但班表未變動，亦無法重新申請
                             ...                        
261    請協助開通秘書室行政助理(專案人員)邱韻娉差勤系統，(AD帳號:yunping-siraya...
262                                  請協助開啟本校公務人員加班餘數試算功能
263                   人員組織樹，技正有重複，而且會看到以前當科長的同仁差單，不知如何解決
264    1/13上午08:53刷卡，因未顯示卡別，致系統出現刷卡不一致，重新轉成出勤資料亦無法處理，...
265          請協助填寫"是否連續在職"、"初任公職日"等黃底之"資料欄位英文名稱"及"資料表名稱"
Name: 摘要, Length: 266, dtype: object

In [80]:
# 取每個主題對應 topN 重要關鍵字
def top_words_data_frame(model: LatentDirichletAllocation,
                         tf_idf_vectorizer: TfidfVectorizer,
                         n_top_words: int) -> pd.DataFrame:
    '''
    求出每个主题的前 n_top_words 个词

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    tf_idf_vectorizer : sklearn 的 TfidfVectorizer
    n_top_words :前 n_top_words 个主题词

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    rows = []
    feature_names = tf_idf_vectorizer.get_feature_names_out()
    for topic in model.components_:
        top_words = [feature_names[i]
                     for i in topic.argsort()[:-n_top_words - 1:-1]]
        rows.append(top_words)
    columns = [f'word {i+1}' for i in range(n_top_words)]
    df = pd.DataFrame(rows, columns=columns)

    return df

In [81]:
# 定義預測函式
def predict_to_data_frame(model: LatentDirichletAllocation, X: np.ndarray) -> pd.DataFrame:
    '''
    求出文档主题概率分布情况

    Parameters
    ----------
    model : sklearn 的 LatentDirichletAllocation 
    X : 词向量矩阵

    Return
    ------
    DataFrame: 包含主题词分布情况
    '''
    matrix = model.transform(X)
    columns = [f'P(topic {i+1})' for i in range(len(model.components_))]
    df = pd.DataFrame(matrix, columns=columns)
    return df



## word tokenization

In [82]:
# 去除重複、去除缺失、分词, 轉繁體
df['abstrat'] = (
    df['摘要']
    .apply(lambda x: str(x))
    .apply(lambda x: re.sub(pattern, ' ', x))
    # .apply(lambda x: re.sub(stop, ' ', x))
    .apply(lambda x: " ".join(jieba.lcut(x, use_paddle=True)))
    .apply(lambda x: cc.convert(x))
)



In [83]:
df[["摘要", "abstrat"]]

,摘要,abstrat
0,人事室主任郭燕燕要查看輪班相關資料，經查詢無資料，畫面如附件,人事 室主任 郭燕燕要 查看 輪班 相關 資料 ， 經查詢 無資料 ， 畫面 如 附件
1,有關北嶺國小112年寒暑休設定,有關 北嶺國 小 112 年 寒暑 休設定
2,秘書室新進人員無法選取該室職稱,秘書室 新 進人員 無法 選取 該室 職稱
3,1/3及1/4曠職部分麻煩請處理，因該員原任職台東地院1/3辭職，1/12本院報到。,1 / 3 及 1 / 4 曠職 部分 麻煩 請 處理 ， 因該 員 原任 職臺 東地院 1...
4,1/10請假，值班表設定帶頒給陳妘綺，申請完成但班表未變動，亦無法重新申請,1 / 10 請假 ， 值班 表設定 帶 頒給 陳 妘 綺 ， 申請 完成 但班表未 變動 ...
...,...,...
261,請協助開通秘書室行政助理(專案人員)邱韻娉差勤系統，(AD帳號:yunping-siraya...,請 協助 開通 秘書室 行政助理 ( 專案 人員 ) 邱韻 娉 差勤 系統 ， ( AD 帳...
262,請協助開啟本校公務人員加班餘數試算功能,請 協助 開啟 本校 公務人員 加班 餘數 試算 功能
263,人員組織樹，技正有重複，而且會看到以前當科長的同仁差單，不知如何解決,人員 組織 樹 ， 技正 有重 複 ， 而且 會 看到 以前 當科長 的 同仁 差單 ， 不...
264,1/13上午08:53刷卡，因未顯示卡別，致系統出現刷卡不一致，重新轉成出勤資料亦無法處理，...,1 / 13 上午 08 : 53 刷卡 ， 因未 顯示 卡別 ， 致系統 出現 刷卡 不 ...


## TF IDF vector

In [84]:
# built tf-idf
tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(df['abstrat'])


In [85]:
# eigen vector list
feature_names = tf_idf_vectorizer.get_feature_names_out()

# TF-IDF matirx
matrix = tf_idf.toarray()
feature_names_df = pd.DataFrame(matrix, columns=feature_names)

# print(feature_names_df)
feature_names_df



,00,01,02,02316,03,05,0600,0750,08,0800,...,體育室,高中,高雄市,麻煩,麻煩查,點到,點完,點核,點選,龐雜
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.243417,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
263,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.242532,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


## LDA model instance

In [86]:
lda = LatentDirichletAllocation(
    n_components=n_topics,
    max_iter=50, # 疊代次數 類比epoch
    learning_method='online', # 優化方式，在線變分推理
    learning_offset=50, # learning rate
    random_state=42 # 亂數種子
    )

In [87]:
# use tf_idf corpus train lda model
lda.fit(tf_idf)

LatentDirichletAllocation(learning_method='online', learning_offset=50,
                          max_iter=50, n_components=3, random_state=42)

## word distribution in topics

In [88]:
# calculate n_top_words topic word
top_words_df = top_words_data_frame(lda, tf_idf_vectorizer, n_top_words)

In [111]:
top_words_df

,word 1,word 2,word 3,word 4,word 5,word 6,word 7,word 8,word 9,word 10,...,word 1491,word 1492,word 1493,word 1494,word 1495,word 1496,word 1497,word 1498,word 1499,word 1500
0,加班,時數,補休,核算,紀錄,出差,試算,功能,系統後臺,給予,...,剩餘,職人員,taitung,日依,吳政益,篩選,二股,彈性,我將,總務長
1,流程,主任,主管,批核,假單,代理,業務,人事,修改,單位,...,容易,問是,先將,依據,人員補,各自,以分,有時,函示,缺勤
2,人員,資料,協助,謝謝,112,無法,差勤,刷卡,上班,系統,...,經廠,補送,人員資料,航務組,之表單,總務,僱運動,音樂學系,助理,有待


## Model prediction

In [90]:
# total features
len(tf_idf_vectorizer.get_feature_names_out())

1653

In [91]:
# tf-idf metrix
# convert tf_idf into tuple，for future calculation by probablity in corpus
X = tf_idf.toarray()
X.shape

(266, 1653)

In [123]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## topic probability in circumstance

In [92]:
# topic probability in circumstance
predict_df = predict_to_data_frame(lda, X)

In [122]:
predict_df

,P(topic 1),P(topic 2),P(topic 3)
0,0.083671,0.708666,0.207663
1,0.107747,0.107236,0.785017
2,0.104128,0.114912,0.780960
3,0.834865,0.079073,0.086062
4,0.710055,0.085323,0.204622
...,...,...,...
261,0.073792,0.073465,0.852742
262,0.818649,0.089921,0.091430
263,0.073990,0.074002,0.852008
264,0.066134,0.065374,0.868492


## topic modeling visualiztion

In [93]:
# pyLDAvis plot 
data = pyLDAvis.lda_model.prepare(lda, tf_idf, tf_idf_vectorizer)
pyLDAvis.save_html(data, html_path)
# clear screen
os.system('clear')
# by browser open html doc to check result 
os.system(f'start {html_path}')


print("--" * 20)
print('本次生成了文件：',
      top_words_csv_path,
      predict_topic_csv_path,
      html_path)

----------------------------------------
本次生成了文件： poetry_topic_modeling.csv poetry-distribution.csv poetry-lda-visualization.html


## save file

In [ ]:
# save as n_top_words to csv
# top_words_df.to_csv(top_words_csv_path, encoding='utf-8-sig', index=None)

# # save predict topic to csv
# predict_df.to_csv(predict_topic_csv_path, encoding='utf-8-sig', index=None)

## Interpretation of Topic Modeling Metrics

1. saliency(term w) = frequency(w) * [sum_t p(t | w) * log(p(t | w)/p(t))] for topics t; see Chuang et. al (2012)
2. relevance(term w | topic t) = λ * p(w | t) + (1 - λ) * p(w | t)/p(w); see Sievert & Shirley (2014)


### 1. Saliency (顯著性) -> blue bar
- `saliency(term w)` measures the importance of term \( w \) across a set of topics.
- `frequency(w)` is the frequency of term \( w \) in the entire corpus.
- \( \sum_t p(t | w) \) sums the probabilities of all possible topics \( t \) given term \( w \).
- \( \log(p(t | w)/p(t)) \) is the log of the ratio of the probability of topic \( t \) given term \( w \) to the marginal probability of topic \( t \) across all documents.
- This metric calculates the saliency of a term by multiplying its frequency in a specific topic with the difference in its distribution across all topics.

### 2. Relevance (相關性) -> red bar
- `relevance(term w | topic t)` measures the relevance of term \( w \) within a specific topic \( t \).
- λ (`λ`) is a parameter between 0 and 1 used to balance the probability of term \( w \) in topic \( t \) \( p(w | t) \) with the lift of the term \( p(w | t)/p(w) \).
- `p(w | t)` is the probability of term \( w \) occurring within topic \( t \).
- `p(w)` is the marginal probability of term \( w \) occurring across all documents.
- This formula measures relevance by combining the conditional probability of a term in a specific topic with its lift across the corpus.

Overall, these formulas are used to determine the significance of terms within specific topics, which is crucial for understanding and interpreting the results of topic models. These metrics help in selecting terms that are both common and distinctive to a topic, enhancing our understanding of the topic's meaning.


## check

In [126]:
## 包含管理關鍵字
df[df["abstrat"].str.contains('管理', case=False)]["abstrat"]


10     請 問系統 管理 - 機關 差勤 - 加班 時數 限制 ， 為何 沒有 一般 加班 平日 可...
13     配合 公務員 服務法 修正 ， 平日 加班 時數 上限 為 4 小時 ， 但系統 管理 / ...
30     本校 人事 人員業 完成 112 年 寒假 的 [ 差勤 管理 / 寒暑 休設 定維護 ] ...
43     莒光 站 反應 要 進差 勤系統 請 出差 或 加班 時 網頁 都 無法 馬 上 進去 ， ...
86     因為 學校 寒暑假 有 不同 差勤 規定 ， 請問 如何 一次 調整 多人 差勤 規定 ？ ...
100    中辦 支付 管理 組林麗華 的 代理人 通常 會 設給 中辦 的 菸酒 組陳長 安 ， 以前...
108    主計處 薪資 管理系 統之年終 、 考績 AKM 檔 成功 匯入 表單 簽核 系統 ， 但 ...
123    因 職務輪調 ， 更換 請 購單 管理 人員 ， 原本 請 購單 流程 是 申請 人 - 申...
144    鍾 正光 專委 、 程泰源 專委 於 今年 1 月 16 日屆齡 退休 ， 原已 預設 好 ...
152    煩請 協助 刪除 刷卡 時間 單位 ： 行政 管理 組 姓名 ： 廖佳怡 刷卡 時間 ： 1...
188    今 ( 16 ) 日要 在 「 系統 管理 」 → 「 人員 管理 」 → 「 人員 基本 ...
192    您好 ， 有關 之 前提 問 加班 餘數合 併 設定 ， 業以 設定 完成 ， 因協助 同仁...
211         系統 管理者 、 差勤 管理 功能 選項 消失 了 ( 差勤 管理 、 工具 ... )
223    林員身 為 南化 管理站 主任 ， 可以 點核 所屬 人員 差勤 ， 但 卻 在 其他 所屬...
230    陳 美靜 為 曾文 管理站 主任 ， 遊憩 課 張麗君 為 遊憩 課 代理 課長 及 秘書室...
247    林宏緯為 工務課 課長 ， 但 他 的 所屬 差勤 資料 卻 看到 是 曾文 管理站 的 人...
Name: abstrat, dtype: object